In [1]:
import re
import pandas as pd
import numpy as np

In [3]:
issues_df = pd.read_csv('./data/gokube_phase1_jun19/GH_cve_labeled_issues.csv')
prs_df = pd.read_csv('./data/gokube_phase1_jun19/GH_cve_labeled_prs.csv')

issues_df['issue_title'] = issues_df.issue_title.fillna('')
issues_df['issue_body'] = issues_df.issue_body.fillna('')
issues_df['description'] = issues_df['issue_title'].map(str) + ' ' + issues_df['issue_body']

prs_df['pr_title'] = prs_df.pr_title.fillna('')
prs_df['pr_body'] = prs_df.pr_body.fillna('')
prs_df['description'] = prs_df['pr_title'].map(str) + ' ' + prs_df['pr_body']

df = pd.concat([issues_df[['description', 'class_label']], 
                prs_df[['description', 'class_label']]], axis=0).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152151 entries, 0 to 152150
Data columns (total 2 columns):
description    152151 non-null object
class_label    152151 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


In [4]:
df.class_label.value_counts()

0    151480
2       671
Name: class_label, dtype: int64

In [5]:
def security_vulnerability_labeler(issue_description):
    strong_vuln_pattern = ('(?i)(advisory|attack|(un)?authoriz(e|ation)|'
                           'clickjack|cross.site|csrf|\\bCVE.*?\\b|deadlock|'
                           'denial.of.service|\\bEOP\\b|exploit|hijack|'
                           'infinite.loop|malicious|\\bNVD\\b|OSVDB|'
                           '\\bRCE\\b|\\bReDoS\\b|\\bDDoS\\b|remote.code.execution|'
                           'security|victim|\\bvuln|\\bXEE\\b|\\bXSRF\\b|'
                           '\\bXSS\\b|\\bXXE\\b)')

    medium_vuln_pattern = ('(?i)(authenticat(e|ion)|brute force|bypass|'
                           'constant.time|crack|credential|\\bDoS\\b|'
                           'expos(e|ing)|hack|harden|injection|lockout|'
                           'overflow|password|\\bPoC\\b|proof.of.concept|'
                           'poison|priv(ilege|elege|elage|lage)|\\b(in)?secur(e|ity)|'
                           '(de)?serializ|spoof|timing|traversal)')

    low_vuln_pattern = ('(?i)(abuse|compliant|constant.time|credential|\\bcrypto|'
                        'escalate|exhaustion|forced|infinite|RFC\\d{4,5})')
    
    if (re.findall(low_vuln_pattern, issue_description, re.I) or
        re.findall(medium_vuln_pattern, issue_description, re.I) or
        re.findall(strong_vuln_pattern, issue_description, re.I)):
        return 1
    else:
        return 0

In [6]:
negative_df = df[df.class_label == 0]
positive_df = df[df.class_label == 2]

In [13]:
negative_df['label'] = negative_df.apply(lambda row: security_vulnerability_labeler(row.description), axis=1)

/home/redanalyze/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
positive_df['label'] = positive_df['class_label']

/home/redanalyze/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
df = pd.concat([positive_df[['description', 'label']],
                negative_df[['description', 'label']]], axis=0).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152151 entries, 0 to 152150
Data columns (total 2 columns):
description    152151 non-null object
label          152151 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


In [16]:
df['label'].value_counts()

0    128908
1     22572
2       671
Name: label, dtype: int64

In [17]:
df.to_csv('./data/gokube_phase1_jun19/GH_complete_labeled_issues_prs.csv', index=False)